In [61]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from zipfile import ZipFile

def process_zip_file(zip_path, specified_wells, start_concentration, output_csv_path, heatmap_png_path):
    """
    Extracts, processes data from specified wells, and outputs a CSV and PNG heatmap.
    
    Args:
        zip_path (str): Path to the ZIP file.
        specified_wells (list): List of wells to process.
        start_concentration (float): Starting concentration for the first well.
        output_csv_path (str): Path to save the processed CSV.
        heatmap_png_path (str): Path to save the PNG heatmap.
    """
    # Extract ZIP file
    with ZipFile(zip_path, 'r') as zip_ref:
        extract_path = os.path.splitext(zip_path)[0]
        zip_ref.extractall(extract_path)

    files = [os.path.join(extract_path, f"Well_{well}.csv") for well in specified_wells if os.path.exists(os.path.join(extract_path, f"Well_{well}.csv"))]
    print(files)
    combined_data = pd.DataFrame()
    start_time = None

    for file in files:
        data = pd.read_csv(file)
        data['Timestamp'] = pd.to_datetime(data['Timestamp'])
        
        if start_time is None:
            start_time = data['Timestamp'].min()

        data['HoursFromStart'] = (data['Timestamp'] - start_time).dt.total_seconds() / 3600
        data['TimeBin'] = data['HoursFromStart'].apply(lambda x: int(x))  # Group by 1-hour increments

        data = data[data['noise'] >= 0.7]
        # remove any values with 02less than -100 and greater than 200
        data = data[data['O2 (percent air)'] >= -100]
        data = data[data['O2 (percent air)'] <= 200]
        avg_data = data.groupby('TimeBin')['O2 (percent air)'].mean().reset_index()

        well_number = int(file.split('_')[3].split('.')[0])
        print(f'Processing Well {well_number}...')
        division_count = well_number // 49
        concentration = start_concentration / (3 ** division_count)

        
        avg_data['Concentration'] = concentration
        avg_data['Well_Number'] = well_number
        combined_data = pd.concat([combined_data, avg_data], ignore_index=True)

    
    # Drop duplicate combinations
    combined_data = combined_data.drop_duplicates(subset=['Well_Number', 'Concentration', 'TimeBin'])

    # Pivot for heatmap format
    heatmap_data = combined_data.pivot(index=['Well_Number', 'Concentration'], columns='TimeBin', values='O2 (percent air)')

    heatmap_data.to_csv(output_csv_path)

    # Create and save heatmap
    plt.figure(figsize=(20, 10))
    sns.heatmap(heatmap_data, cmap="viridis", annot=False)
    plt.title('O2 Concentration Heatmap by Well and Hourly Increments')
    plt.xlabel('TimeBin')
    plt.ylabel('Well/Concentration')
    plt.tight_layout()
    plt.savefig(heatmap_png_path)
    plt.close()



In [66]:
# Example usage:
zip_input_path = "External Data/Wells_172_Pl1.zip"
# derive the well numbers from a 384-well plate (16x24) for columns 23 and 24
specified_wells = [23 + 24 * i for i in range(16)] + [24 + 24 * i for i in range(16)]
start_concentration = 22.5
output_csv_path = "output.csv"
heatmap_png_path = "heatmap.png"
process_zip_file(zip_input_path, specified_wells, start_concentration, output_csv_path, heatmap_png_path)


['External Data/Wells_172_Pl1/Well_23.csv', 'External Data/Wells_172_Pl1/Well_47.csv', 'External Data/Wells_172_Pl1/Well_71.csv', 'External Data/Wells_172_Pl1/Well_95.csv', 'External Data/Wells_172_Pl1/Well_119.csv', 'External Data/Wells_172_Pl1/Well_143.csv', 'External Data/Wells_172_Pl1/Well_167.csv', 'External Data/Wells_172_Pl1/Well_191.csv', 'External Data/Wells_172_Pl1/Well_215.csv', 'External Data/Wells_172_Pl1/Well_239.csv', 'External Data/Wells_172_Pl1/Well_263.csv', 'External Data/Wells_172_Pl1/Well_287.csv', 'External Data/Wells_172_Pl1/Well_311.csv', 'External Data/Wells_172_Pl1/Well_335.csv', 'External Data/Wells_172_Pl1/Well_359.csv', 'External Data/Wells_172_Pl1/Well_383.csv', 'External Data/Wells_172_Pl1/Well_24.csv', 'External Data/Wells_172_Pl1/Well_48.csv', 'External Data/Wells_172_Pl1/Well_72.csv', 'External Data/Wells_172_Pl1/Well_96.csv', 'External Data/Wells_172_Pl1/Well_120.csv', 'External Data/Wells_172_Pl1/Well_144.csv', 'External Data/Wells_172_Pl1/Well_168.c